In [18]:
from MLForecastPipeline import *

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

LAG_TRANSFORMS_MAP = {
    "expanding_mean_rolling_14_rolling_30": {1: 'expanding_mean', 7: 'expanding_mean', 30: 'rolling_mean_30'},
    "expanding_mean_rolling_14": {1: 'expanding_mean', 7: 'rolling_mean_14', 30: 'expanding_mean'},
    "rolling_14_rolling_30_expanding": {1: 'rolling_mean_14', 7: 'rolling_mean_30', 30: 'expanding_mean'},
    "rolling_14_expanding": {1: 'rolling_mean_14', 30: 'expanding_mean'},
    "rolling_14_only": {1: 'rolling_mean_14'},
    "no_transform": {},
}

def map_lag_transforms(lag_transform_dict, lag_transforms_map=LAG_TRANSFORMS_MAP):
    for name, transform in lag_transforms_map.items():
        if lag_transform_dict == transform:
            return name
    return "unknown"

def analyze_results(df, lag_transforms_map=LAG_TRANSFORMS_MAP, mape_threshold=40, model_filter=None):
    df = df.copy()
    df['Lag Transform Name'] = df['Lag Transforms'].apply(lambda x: map_lag_transforms(x, lag_transforms_map))
    df['Lag_Set_Name'] = df['Lag Name']
    # Identify MAPE columns dynamically
    mape_columns = [col for col in df.columns if col.startswith("test_") and col.endswith("_days")]
    
    # Compute mean MAPE across all test periods
    df['MAPE'] = df[mape_columns].mean(axis=1)
    
    # Apply filtering
    top_df = df[df["MAPE"] < mape_threshold].copy()
    if model_filter:
        top_df = top_df[top_df['Model'] == model_filter].copy()
    
    # Compute groupings
    top_models = top_df.groupby("Model")["MAPE"].mean().sort_values().reset_index()
    top_transforms = top_df.groupby("Transforms")["MAPE"].mean().sort_values().reset_index()
    top_lag_transforms = top_df.groupby("Lag Transform Name")["MAPE"].mean().sort_values().reset_index()
    top_lags = top_df.groupby("Lag_Set_Name")["MAPE"].mean().sort_values().reset_index()
    
    # Compute MAPE trends over different forecasting horizons
    mape_trends = top_df.groupby("Model")[mape_columns].mean().reset_index()
    
    return top_models, top_transforms, top_lag_transforms, top_lags, mape_trends

def plot_results(top_models, top_transforms, top_lag_transforms, top_lags, mape_trends):
    plt.figure(figsize=(12, 5))
    sns.barplot(x=top_models["Model"], y=top_models["MAPE"], palette="viridis", hue=top_models["Model"])
    plt.xticks(rotation=45)
    plt.title("Average MAPE per Model")
    plt.show()

    plt.figure(figsize=(12, 5))
    sns.barplot(x=top_transforms["Transforms"], y=top_transforms["MAPE"], palette="coolwarm", hue=top_transforms["Transforms"])
    plt.xticks(rotation=90)
    plt.title("Average MAPE per Transform")
    plt.show()

    plt.figure(figsize=(12, 5))
    sns.barplot(x=top_lag_transforms["Lag Transform Name"], y=top_lag_transforms["MAPE"], palette="Blues", hue=top_lag_transforms["Lag Transform Name"])
    plt.xticks(rotation=90)
    plt.title("Average MAPE per Lag Transform")
    plt.show()

    plt.figure(figsize=(12, 5))
    sns.barplot(x=top_lags["Lag_Set_Name"], y=top_lags["MAPE"], palette="Blues", hue=top_lags["Lag_Set_Name"])
    plt.xticks(rotation=90)
    plt.title("MAPE vs Number of Lags")
    plt.show()
    
    # Plot MAPE trends across different forecasting horizons
    plt.figure(figsize=(12, 5))
    for model in mape_trends["Model"]:
        plt.plot(mape_trends.columns[1:], mape_trends[mape_trends["Model"] == model].values[0][1:], label=model)
    plt.xlabel("Forecasting Horizon (Days)")
    plt.ylabel("MAPE")
    plt.title("MAPE Trends Across Forecast Horizons")
    plt.legend()
    plt.show()

# Example usage:
# top_models, top_transforms, top_lag_transforms, top_lags, mape_trends = analyze_results(df, lag_transforms_map, optimal_lags_map)
# plot_results(top_models, top_transforms, top_lag_transforms, top_lags, mape_trends)


In [4]:
# import pandas as pd
# import numpy as np
# import glob
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Load all results
# results = {}
# for file in glob.glob("results/run_3/*.csv"):
#     dataset_name = file.split("/")[-1].replace(".csv", "")
#     results[dataset_name] = pd.read_csv(file)

# # Combine all datasets into a single DataFrame
# df = pd.concat([df.assign(Dataset=name) for name, df in results.items()], ignore_index=True)

# # Define threshold for MAPE
# mape_threshold = 35  

# # Identify best models per test period
# best_models_per_period = {}
# for test_n in [30, 60, 90, 120, 150, 180, 240, 300, 360, 480, 600, 720, 737]:
#     col = f"test_{test_n}_days"
#     best_models_per_period[test_n] = df[df[col] < mape_threshold].nsmallest(1, col)

# # Calculate variance across all test columns
# test_cols = [col for col in df.columns if "test_" in col]
# df["stability"] = df[test_cols].std(axis=1)

# # Select most stable model
# most_stable_model = df.nsmallest(1, "stability")

In [2]:
import glob
results = {}
for file in glob.glob("results/run_3/*.csv"):
    dataset_name = file.split("\\")[-1].replace(".csv", "")
    results[dataset_name] = pd.read_csv(file)

In [ ]:
name_parts = ['123', '456', '567', '34']


'567_34'

In [14]:
import re

def extract_train_info(dataset_name):
    """
    Extracts sensor ID, training length in months, and a standardized train label.
    
    Example Inputs:
        - "6_NH_15D_train"  → (6, 0.5, "NH_15D_train")
        - "6_H_5M_train"    → (6, 5, "H_5M_train")
        - "2_18M_train"     → (2, 18, "18M_train")
    
    Returns:
        - sensor (int): Sensor ID
        - train_months (float): Training length in months
        - train_label (str): Everything after the sensor ID (used for finding comparable datasets)
    """
    name_parts = dataset_name.split('_')

    # Extract sensor ID
    sensor = int(name_parts[0])  # First part is always the sensor number

    # Reconstruct the label for easy dataset comparison
    train_label = '_'.join(name_parts[1:])

    # Extract training length (2nd to last part contains number + unit)
    train_info = name_parts[-2]  # Example: "15D" or "5M"
    match = re.match(r"(\d+)([MD])", train_info)  # Extract number and unit

    if match:
        train_length = int(match.group(1))
        unit = match.group(2)

        # Convert days to months (approximate)
        train_months = train_length / 30 if unit == "D" else train_length
    else:
        return None, None, None  # Invalid format, return None values

    return sensor, train_months, train_label

# Example Usage
datasets = ["6_NH_15D_train", "6_H_5M_train", "2_18M_train"]
for ds in datasets:
    print(f"{ds} → {extract_train_info(ds)}")


6_NH_15D_train → (6, 0.5, 'NH_15D_train')
6_H_5M_train → (6, 5, 'H_5M_train')
2_18M_train → (2, 18, '18M_train')


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

# Define parameters
HORIZONS = [30, 60, 90, 120, 150, 180, 240, 300, 360, 480, 600, 720, 737]
MAPE_COLUMNS = [f"test_{h}_days" for h in HORIZONS]

# Thresholds for different filtering criteria
EARLY_HORIZON_THRESHOLD = 30  # MAPE threshold for early horizons
REFERENCE_THRESHOLD = 30       # Threshold for comparison with other sensors
ONE_THIRD_THRESHOLD = 30       # Threshold for MAPE at 1/3 of training length

EARLY_HORIZONS = [30, 60, 90, 120]  # Horizons we check for early filtering
SENSORS_TO_COMPARE = [2, 5, 6]  # Sensors that share training length criteria

# Process each dataset
for dataset_name, df in results.items():
    print("=" * 80)
    print(f"DATASET: {dataset_name}")
    print("=" * 80)

    sensor_id, train_months, train_label = extract_train_info(dataset_name)
    if sensor_id is None:
        print("Skipping dataset: Cannot determine sensor ID or train length.")
        continue

    mape_columns_local = [c for c in MAPE_COLUMNS if c in df.columns]

    # --- (a) Filtering Criteria ---

    # 1. Remove models exceeding the early horizon threshold
    early_horizon_cols = [f"test_{h}_days" for h in EARLY_HORIZONS if f"test_{h}_days" in df.columns]
    mask_early_horizon = (df[early_horizon_cols] <= EARLY_HORIZON_THRESHOLD).all(axis=1)

    # TODO
    # 2. Find comparable sensors (same train period but different sensor)
    comparable_sensors = [f"{sensor}_{train_label}" for sensor in SENSORS_TO_COMPARE if sensor != sensor_id]
    reference_mape = None
    for comp_sensor in comparable_sensors:
        if comp_sensor in results:
            reference_mape = results[comp_sensor][mape_columns_local].mean().mean()  # Avg MAPE for reference
            break  # Take the first matching sensor

    if reference_mape:
        mask_reference_sensors = df[mape_columns_local].mean(axis=1) <= REFERENCE_THRESHOLD
    else:
        mask_reference_sensors = True  # Skip filter if no reference data

    # 3. Ensure MAPE is good at 1/3 of train length
    one_third_horizon = int(train_months * 30 // 3)  # Convert months to days
    closest_horizon = min(HORIZONS, key=lambda x: abs(x - one_third_horizon))  # Find closest matching horizon

    if f"test_{closest_horizon}_days" in df.columns:
        mask_one_third = df[f"test_{closest_horizon}_days"] <= ONE_THIRD_THRESHOLD
    else:
        mask_one_third = True  # Skip if column isn't available

    # Apply filters
    df_filtered = df[mask_early_horizon & mask_reference_sensors & mask_one_third].copy()

    # --- (b) Compute stability metrics ---
    df_filtered["std_mape"] = df_filtered[mape_columns_local].std(axis=1)
    df_filtered["avg_mape"] = df_filtered[mape_columns_local].mean(axis=1)

    # --- (c) Identify best model for each horizon ---
    best_models_each_horizon = {}
    for horizon_col in mape_columns_local:
        if not df_filtered.empty and df_filtered[horizon_col].notna().sum() > 0:
            idx_min = df_filtered[horizon_col].idxmin()
            best_models_each_horizon[horizon_col] = df_filtered.loc[idx_min]

    # Print out best models per horizon
    print("Best Model per Horizon (filtered models):")
    for horizon_col, row_data in best_models_each_horizon.items():
        model_name = row_data["Model"]
        transforms = row_data["Transforms"]
        lags = row_data["Lags"]
        value = row_data[horizon_col]
        print(f"  {horizon_col}: {model_name} with MAPE={value:.2f}, transforms={transforms}, lags={lags}")

    # --- (d) Identify best stable model overall ---
    if not df_filtered.empty:
        best_stability_idx = df_filtered["std_mape"].idxmin()
        best_stability_row = df_filtered.loc[best_stability_idx]
        print("\nMost Stable Model (lowest std of MAPE across horizons, filtered models):")
        print("  Model:     ", best_stability_row["Model"])
        print("  STD MAPE:  ", f"{best_stability_row['std_mape']:.2f}")
        print("  AVG MAPE:  ", f"{best_stability_row['avg_mape']:.2f}")

DATASET: 2_10M_train
Best Model per Horizon (filtered models):
DATASET: 2_12M_train
Best Model per Horizon (filtered models):
  test_30_days: Lasso with MAPE=18.85, transforms=AutoDifferences(max_diffs=188), lags=[1, 3, 4, 5, 6, 7, 8, 9, 14, 15, 16, 17, 25, 29, 33]
  test_60_days: Ridge with MAPE=21.18, transforms=AutoDifferences(max_diffs=188) | LocalStandardScaler(stats_=[[1.95070557e-02 2.04360268e+01]]), lags=[1, 3, 4, 5, 6, 7, 8, 9, 14, 15, 16, 17, 25, 29, 33]
  test_90_days: Ridge with MAPE=19.84, transforms=AutoDifferences(max_diffs=188) | LocalStandardScaler(stats_=[[1.95070557e-02 2.04360268e+01]]), lags=[1, 3, 4, 5, 6, 7, 8, 9, 14, 15, 16, 17, 25, 29, 33]
  test_120_days: Ridge with MAPE=19.28, transforms=AutoDifferences(max_diffs=188) | LocalStandardScaler(stats_=[[1.95070557e-02 2.04360268e+01]]), lags=[1, 3, 4, 5, 6, 7, 8, 9, 14, 15, 16, 17, 25, 29, 33]
  test_150_days: Ridge with MAPE=18.87, transforms=AutoDifferences(max_diffs=188) | LocalStandardScaler(stats_=[[1.950705